In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Change working directory to project root if currently in notebooks
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

In [16]:
file_path = os.path.join("data", "raw", "churn.csv")

In [17]:
def load_data(raw_path, output_path):
    """Load data from a CSV file."""
    if not os.path.exists(raw_path):
        raise FileNotFoundError(f"The file {raw_path} does not exist.")
    
    print(f"Loading data from {raw_path}...")
    data = pd.read_csv(raw_path)

    data.columns = [col.lower().replace(" ", "_") for col in data.columns]

    data['totalcharges'] = pd.to_numeric(data['totalcharges'], errors='coerce')
    data['totalcharges'] = data['totalcharges'].fillna(0)

    if 'churn' in data.columns:
        data['churn'] = data['churn'].map({'Yes': 1, 'No': 0})

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    data.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}.")

In [18]:
input_path = os.path.join("data", "raw", "customer_churn.csv")
output_train_path = os.path.join("data", "processed", "train.csv")
output_test_path = os.path.join("data", "processed", "test.csv")

In [19]:
def process_data(input_path, output_train_path, output_test_path):
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"The file {input_path} does not exist.")
    
    print(f"Loading data from {input_path}...")
    data = pd.read_csv(input_path)

    if 'customerid' in data.columns:
        data = data.drop('customerid', axis=1)

    data = pd.get_dummies(data, drop_first=True)

    X = data.drop('churn', axis=1)
    y = data['churn']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    train_data.to_csv(output_train_path, index=False)
    test_data.to_csv(output_test_path, index=False)

    print(f"Training data saved to {output_train_path}.")

In [22]:
load_data(file_path, input_path)
process_data(input_path, output_train_path, output_test_path)

Loading data from data/raw/churn.csv...
Data saved to data/raw/customer_churn.csv.
Loading data from data/raw/customer_churn.csv...
Training data saved to data/processed/train.csv.


In [23]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import os

In [27]:
def train():
    # 1. Load dữ liệu đã chuẩn bị ở bước trước
    print("🚀 Đang load dữ liệu train/test...")
    train_df = pd.read_csv(output_train_path)
    test_df = pd.read_csv(output_test_path)
    
    # Tách Feature (X) và Target (y)
    # Target là cột 'churn', các cột còn lại là Feature
    X_train = train_df.drop('churn', axis=1)
    y_train = train_df['churn']
    X_test = test_df.drop('churn', axis=1)
    y_test = test_df['churn']
    
    # --- BẮT ĐẦU MLFLOW ---
    # set_experiment giúp gom nhóm các lần chạy lại cho gọn
    mlflow.set_experiment("churn-prediction-baseline")
    
    with mlflow.start_run():
        # 2. Định nghĩa tham số Model
        params = {
            "C": 0.01,           # Regularization strength
            "solver": "liblinear",
            "max_iter": 1000
        }
        
        # Log tham số lên MLflow (để sau này nhớ mình đã chỉnh gì)
        mlflow.log_params(params)
        
        # 3. Train Model
        print("🧠 Đang training model...")
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)
        
        # 4. Đánh giá Model
        predictions = model.predict(X_test)
        predict_proba = model.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, predictions)
        auc = roc_auc_score(y_test, predict_proba)
        
        print(f"📊 Kết quả: Accuracy={acc:.4f}, AUC={auc:.4f}")
        
        # Log chỉ số (Metrics) lên MLflow
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("roc_auc", auc)
        
        # 5. Lưu Model vào MLflow
        # Giúp bạn có thể tải lại model này ở bất kỳ đâu
        mlflow.sklearn.log_model(model, "model")
        
        print("✅ Đã log model và metrics lên MLflow!")

In [28]:
train()

2025/11/29 13:45:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🚀 Đang load dữ liệu train/test...
🧠 Đang training model...
📊 Kết quả: Accuracy=0.8148, AUC=0.8616


2025/11/29 13:45:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Đã log model và metrics lên MLflow!


In [ ]:
!mlflow ui

/home/t1nt1n/miniconda3/envs/churn-env/lib/python3.10/site-packages/mlflow/server/handlers.py:256: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, artifact_uri)
/home/t1nt1n/miniconda3/envs/churn-env/lib/python3.10/site-packages/mlflow/server/handlers.py:285: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)
[MLflow] Security middleware enabled with default settings (localhost-only). To allow connections from other hosts, use --host 0.0.0.0 and configure --allowed-hosts and --cors-allowed-origins.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:     Started parent process [141